In [1]:
from llama_index import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from llama_index.llms import Bedrock
from llama_index import ServiceContext, StorageContext, load_index_from_storage
from llama_index.embeddings import BedrockEmbedding
import os.path

/Users/akshayranganath/Projects/idea/yolov5/venv/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [38]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [39]:
PERSIST_DIR = "./storage"

#llm = Bedrock(model='anthropic.claude-v2',temperature=0,profile_name='aws_sol')
llm = Bedrock(model='anthropic.claude-instant-v1',temperature=0,profile_name='aws_sol')
embedding = BedrockEmbedding.from_credentials(aws_profile='aws_sol')
# create a service context using the Bedrock embeddings
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding    
)  

In [42]:
def main(url:str)->None:
    # check if index exists. if not, create one.
    if not os.path.exists(PERSIST_DIR):
        documents = SimpleWebPageReader(html_to_text=True).load_data(urls=[url])
        index = VectorStoreIndex.from_documents(documents,service_context=service_context)
        # now persist the index
        index.storage_context.persist(persist_dir=PERSIST_DIR)
    # load the index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(service_context=service_context, storage_context=storage_context)
    query_engine = index.as_query_engine()
    response = query_engine.query('Are there any action items for a comapny that is embarking on building a foundational model?')
    print(response)

In [43]:
main(url='https://www.whitehouse.gov/briefing-room/statements-releases/2023/10/30/fact-sheet-president-biden-issues-executive-order-on-safe-secure-and-trustworthy-artificial-intelligence/')

 There are several considerations for any organization embarking on foundational AI work. Foremost is establishing guiding principles to develop technologies that are safe, secure, private and beneficial to users. It's also important to involve diverse experts and consider how technologies may impact different communities. Ongoing research and oversight can help address risks, while engagement with stakeholders ensures technologies meet real needs. Overall, foundational models should be developed responsibly and for the benefit of all.


In [2]:
import chromadb
from llama_index.vector_stores import ChromaVectorStore

In [3]:
dbconn = chromadb.Client(chromadb.config.Settings(is_persistent=True, persist_directory='./storage'))
    

In [58]:
dbconn.create_collection('test')

ValueError: Collection test already exists.

In [59]:
dbconn.list_collections()

[Collection(name=test)]

In [60]:
collection = dbconn.get_collection('test')

In [63]:
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world."""

In [64]:
collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

/Users/akshayranganath/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79.3M/79.3M [00:03<00:00, 21.5MiB/s]


In [68]:
results = collection.query(
    query_texts=["What is the student name?"],
    n_results=3
)

results

{'ids': [['id1', 'id2', 'id3']],
 'distances': [[1.2946666443895525, 1.3954031898157733, 1.5187653929866805]],
 'metadatas': [[{'source': 'student info'},
   {'source': 'club info'},
   {'source': 'university info'}]],
 'embeddings': None,
 'documents': [['\nAlexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,\nis a member of the programming and chess clubs who enjoys pizza, swimming, and hiking\nin her free time in hopes of working at a tech company after graduating from the University of Washington.\n',
   "\nThe university chess club provides an outlet for students to come together and enjoy playing\nthe classic strategy game of chess. Members of all skill levels are welcome, from beginners learning\nthe rules to experienced tournament players. The club typically meets a few times per week to play casual games,\nparticipate in tournaments, analyze famous chess matches, and improve members' skills.\n",
   '\nThe University of Washington, founded in 1861 in Sea

## Back to work
Let's get back to indexing our URL

In [87]:
def index_page(url:str)->None:
    # first load the page
    documents = SimpleWebPageReader(html_to_text=True).load_data(urls=[url])
    #print(documents[0].text)
    collection.add(
        documents=[documents[0].text],
        metadatas=[{"source": url}],
        ids = ['id4']
    )

In [88]:
index_page('https://akshayranganath.github.io/Comparing_Animated_Image_Formats/')

In [94]:
resp = collection.query(query_texts=['What are the suggested formats for animated images?'])

This does embedding and storage using the default text embedding algorithm.

Now, getting back to using Bedrock embedding

In [102]:
# import
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import BedrockEmbedding
from IPython.display import Markdown, display
from llama_index.llms import Bedrock
import chromadb

In [134]:
PERSIST_DIR = "./storage"

#llm = Bedrock(model='anthropic.claude-v2',temperature=0,profile_name='aws_sol')
llm = Bedrock(model='anthropic.claude-instant-v1',temperature=0,profile_name='aws_sol')
embedding = BedrockEmbedding.from_credentials(aws_profile='aws_sol')
# create a service context using the Bedrock embeddings
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding    
)  

dbconn = chromadb.Client(chromadb.config.Settings(is_persistent=True, persist_directory='./storage'))
chroma_collection = dbconn.get_collection('url-analyzer') #dbconn.create_collection('url-analyzer')


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [135]:
import uuid

def load_page(url:str)->None:
    # first load the page
    documents = SimpleWebPageReader(html_to_text=True).load_data(urls=[url])
    index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context, service_context=service_context
    )

In [136]:
load_page('https://akshayranganath.github.io/Rate-Limiting-With-Python/')

In [137]:
# now read the data
# load from disk
db2 = chromadb.PersistentClient(path="./storage")
chroma_collection = db2.get_collection('url-analyzer')
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

print(index.vector_store)

stores_text=True is_embedding_query=True flat_metadata=True collection_name=None host=None port=None ssl=False headers=None persist_dir=None collection_kwargs={}


In [140]:
# Query Data from the persisted index
query_engine = index.as_query_engine()
response = query_engine.query("Can you show sample code for rate limiting with python?")

In [143]:
display(Markdown(response.response))

 Here is sample Python code to implement rate limiting:

```python
from ratelimiter import RateLimiter

@RateLimiter(max_calls=30, period=1) 
def call_api():
   # make API call
   pass

for i in range(60):
   call_api() 
```

The @RateLimiter decorator from the ratelimiter library is used to limit the call_api function to 30 calls per minute. This ensures the calls remain within the rate limit even if the code is running concurrently in multiple threads/processes.

In [5]:
PERSIST_DIR = './chat_with_website/storage/'

#llm = Bedrock(model='anthropic.claude-v2',temperature=0,profile_name='aws_sol')
llm = Bedrock(model='anthropic.claude-instant-v1',temperature=0,profile_name='aws_sol')
embedding = BedrockEmbedding.from_credentials(aws_profile='aws_sol')
# create a service context using the Bedrock embeddings
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding    
)  
dbconn = chromadb.Client(chromadb.config.Settings(is_persistent=True, persist_directory='./storage'))
    

dbconn.list_collections()

[Collection(name=test),
 Collection(name=url-analyzer),
 Collection(name=quickstart)]

In [8]:
dbconn.delete_collection(name='quickstart')